In [38]:
%%file load_data.m
%%%%% Load data into variables %%%%%

fprintf('Loading ptis...\n');
ptis = load('out/ptis.csv');

fprintf('Loading keys...\n');
keys = load('out/keys.csv');

fprintf('Loading ctos...\n');
ctos = load('out/ctos.csv');

fprintf('Loading traces...\n');
traces = load('out/traces.csv');

fprintf('Data loaded!\n');

In [39]:
%%file plot_trace.m
%%%%% Plot trace nbr 1 %%%%%
plot(traces(1,:));
xlabel('Time');
ylabel('Amplitude');
title('First trace');
hold on;
xline(360, "--b", "Round 1");
xline(3510, "--b", "Dernier round");
hold off;

In [40]:
%%file plot_average_trace.m
%%%%% Compute and plot average of traces %%%%%
function plot_average_trace(traces)
    avg = mean(traces);
    plot(avg);
    xlabel('Time');
    ylabel('Amplitude');
    title('Average trace');
end

In [41]:
%%file last_round.m
%%%%% Extract last round (2700,3200) %%%%%
function [last_round, avg_last] = last_round(traces)
    % extract last round
    last_round = zeros(20000, 3200 - 2700 + 1);
    for i = 1:20000
        last_round(i,:) = traces(i, 2700:3200);
    end
    avg_last = mean(last_round);
end

In [42]:
%%file generate.m
%%%%% Generate possible states %%%%%
X = zeros(nb_traces, 16);
for k = 1:nb_traces
    X(k, :) = ctos(k, :);
end
gen = uint8(zeros(nb_traces, 256, 16));
gen_xor = uint8(zeros(nb_traces, 256, 16));
gen_shift = uint8(zeros(nb_traces, 256, 16));
gen_sbox = uint8(zeros(nb_traces, 256, 16));

for trace = 1:nb_traces
    for possibility = 1:256
        for val = 1:16
            gen(trace, possibility, val) = uint8(X(trace, val));
        end
    end
end

for trace = 1:nb_traces
    for possibility = 1:256
        for val = 1:16
            gen_xor(trace, possibility, val) = uint8(bitxor(gen(trace, possibility, val), uint8(possibility - 1)));
        end
    end
end

% shift rows
shiftRow = [1 6 11 16 5 10 15 4 9 14 3 8 13 2 7 12];
invShiftRow = [1 14 11 8 5 2 15 12 9 6 3 16 13 10 7 4];

for trace = 1:nb_traces
    for possibility = 1:256
        for val = 1:16
            gen_shift(trace, possibility, val) = gen_xor(trace, possibility, invShiftRow(val));
        end
    end
end

sBox = SBox();
invSBox(sBox(1:256) + 1) = 0:255;
for trace = 1:nb_traces
    for possibility = 1:256
        for val = 1:16
            gen_sbox(trace, possibility, val) = uint8(invSBox(gen_shift(trace, possibility, val) + 1));
        end
    end
end

In [43]:
%%file SBox.m
%%%%% SBox %%%%%
function [sBox] = SBox()
    sBox = [99 124 119 123 242 107 111 197 48 1 103 43 254 215 171 118 202 130 201 125 250 89 71 240 173 212 162 175 156 164 114 192 183 253 147 38 54 63 247 204 52 165 229 241 113 216 49 21 4 199 35 195 24 150 5 154 7 18 128 226 235 39 178 117 9 131 44 26 27 110 90 160 82 59 214 179 41 227 47 132 83 209 0 237 32 252 177 91 106 203 190 57 74 76 88 207 208 239 170 251 67 77 51 133 69 249 2 127 80 60 159 168 81 163 64 143 146 157 56 245 188 182 218 33 16 255 243 210 205 12 19 236 95 151 68 23 196 167 126 61 100 93 25 115 96 129 79 220 34 42 144 136 70 238 184 20 222 94 11 219 224 50 58 10 73 6 36 92 194 211 172 98 145 149 228 121 231 200 55 109 141 213 78 169 108 86 244 234 101 122 174 8 186 120 37 46 28 166 180 198 232 221 116 31 75 189 139 138 112 62 181 102 72 3 246 14 97 53 87 185 134 193 29 158 225 248 152 17 105 217 142 148 155 30 135 233 206 85 40 223 140 161 137 13 191 230 66 104 65 153 45 15 176 84 187 22];
end

In [44]:
%%file hamming_weight_attack.m
%%%%% Hamming weight attack using generated keys %%%%%
hw_vector = [0 1 1 2 1 2 2 3 1 2 2 3 2 3 3 4 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 1 2 2 3 2 3 3 4 2 3 3 4 3 4 4 5 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 2 3 3 4 3 4 4 5 3 4 4 5 4 5 5 6 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 3 4 4 5 4 5 5 6 4 5 5 6 5 6 6 7 4 5 5 6 5 6 6 7 5 6 6 7 6 7 7 8];
hamming = uint8(zeros(nb_traces, 256, 16));
traces_last = extract_last_round(traces);
for j = 1:256
    for k = 1:16
        hamming(:, j, k) = bitxor(gen_sbox(:, j, k), uint8(X(:, k))); % hamming weight of the difference between the generated ctos and the real ctos
    end
end

phi = hw_vector(hamming + 1);

best_candidate = zeros(16, 1);
lastRound = 2700:3200;
figure
sgtitle("Hamming weight correlation")
for k = 1:16
    % Iterate over values and compute correlation to identify the best candidate
    cor = mycorr(double(phi(1:nb_traces, :, shiftRow(k))), double(traces_last(1:nb_traces, :)));

    [max_cor, max_cor_index] = sort(max(abs(cor), [], 2), 'descend');
    best_candidate(k) = max_cor_index(1) - 1; % the best candidate is the one with the highest correlation
    fprintf("best candidate for byte %d is %d\n", k, best_candidate(k))

    %plot result
    subplot(4, 4, k)
    plot(lastRound, cor(:, :))
    title("k=" + num2str(k))
    xlabel('echantillon')
    ylabel('correlation')

end

In [45]:
%%file mycorr.m
function C = mycorr(x,y)
%faster corrcoef for two matrices (octave style)
%only real numbers(?)
%rows ... observations
%columns ... variables

  [xr,xc] = size(x);
  [yr,yc] = size(y);
  assert((xr==yr), 'Matrix row count mismatch');

  x = x - repmat(mean(x,1),xr,1); % remove means
  y = y - repmat(mean(y,1),yr,1);
  C = x'*y; % (n-1)cov(x,y)
  C = C ./ repmat((sqrt(sum(x.^2,1)))',1,yc); % divide by sqrt((n-1)var(x))
  C = C ./ repmat(sqrt(sum(y.^2,1)),xc,1); % divide by sqrt((n-1)var(y))
  % resulting C ... correlation coefficient

  %  C = C ./ (repmat((sqrt(sum(x.^2,1)))',1,yc) .* repmat(sqrt(sum(y.^2,1)),xc,1));
end


In [46]:
%%file main.m
%%%%%%%%%%%%% Main script %%%%%%%%%%%%%
%--- This script is the one to run ---%
clc;
close all;
nb_traces = 20000;
%% load traces
load_data();

%% plot first trace
fprintf(">>> Plotting first trace <<<\n")
plot_trace();
fprintf("-- done --\n")

%% plot avg
fprintf(">>> Plotting average trace <<<\n")
plot_average_trace(traces);
hold on;
xline(1123, "--b", "Round 1");
xline(3131, "--k", "Dernier round");
hold off;
fprintf("-- done --\n")

%% generate possible keys
fprintf(">>> Computing possible keys <<<\n")
generate();
fprintf("-- done --\n")

%% hamming weight attack
fprintf(">>> Hamming weight attack <<<\n")
hamming_weight_attack();
fprintf("-- done --\n")

%% key to get
fprintf("==============\n")
fprintf(">>> Attack result <<<\n")

disp("key to get is ")
key = '4C8CDF23B5C906F79057EC7184193A67';
disp(key)
key_dec = zeros(16, 1);
for i = 1:16
    key_dec(i) = hex2dec(key((2*i)-1 : 2*i));
end

w = uint8(zeros(11, 4, 4));
w(1, :, :) = reshape(key_dec, 4, 4);

for i = 1:10
    w(i+1, :, :) = key_schu(squeeze(w(i, :, :)), i);
end

key_to_get = squeeze(w(11, :, :));
disp("key to get is ")
disp(key_to_get)

disp("best candidate is ")
disp(reshape(best_candidate, 4, 4))

disp("matching values :")
disp(sum(sum(key_to_get == reshape(best_candidate, 4, 4))))

In [47]:
%%file Subbytes.m
%%%%%%%%%%%%% Subbytes %%%%%%%%%%%%%
function sboxees = Subbytes(pti)

ptibin=dec2bin(pti,8);
sboxes=dec2bin(255);
switch(ptibin)

	% first row
	case '00000000'
 sboxes= '01100011';
	case '00000001'
 sboxes= '01111100';
	case '00000010'
 sboxes= '01110111';
	case '00000011'
 sboxes= '01111011';
	case '00000100'
 sboxes= '11110010';
	case '00000101'
 sboxes= '01101011';
	case '00000110'
 sboxes= '01101111';
	case '00000111'
 sboxes= '11000101';
	case '00001000'
 sboxes= '00110000';
	case '00001001'
 sboxes= '00000001';
	case '00001010'
 sboxes= '01100111';
	case '00001011'
 sboxes= '00101011';
	case '00001100'
 sboxes= '11111110';
	case '00001101'
 sboxes= '11010111';
	case '00001110'
 sboxes= '10101011';
	case '00001111'
 sboxes= '01110110';
	%second row
	case '00010000'
 sboxes= '11001010';
	case '00010001'
 sboxes= '10000010';
	case '00010010'
 sboxes= '11001001';
	case '00010011'
 sboxes= '01111101';
	case '00010100'
 sboxes= '11111010';
	case '00010101'
 sboxes= '01011001';
	case '00010110'
 sboxes= '01000111';
	case '00010111'
 sboxes= '11110000';
	case '00011000'
 sboxes= '10101101';
	case '00011001'
 sboxes= '11010100';
	case '00011010'
 sboxes= '10100010';
	case '00011011'
 sboxes= '10101111';
	case '00011100'
 sboxes= '10011100';
	case '00011101'
 sboxes= '10100100';
	case '00011110'
 sboxes= '01110010';
	case '00011111'
 sboxes= '11000000';
	%third row
	case '00100000'
 sboxes= '10110111';
	case '00100001'
 sboxes= '11111101';
	case '00100010'
 sboxes= '10010011';
	case '00100011'
 sboxes= '00100110';
	case '00100100'
 sboxes= '00110110';
	case '00100101'
 sboxes= '00111111';
	case '00100110'
 sboxes= '11110111';
	case '00100111'
 sboxes= '11001100';
	case '00101000'
 sboxes= '00110100';
	case '00101001'
 sboxes= '10100101';
	case '00101010'
 sboxes= '11100101';
	case '00101011'
 sboxes= '11110001';
	case '00101100'
 sboxes= '01110001';
	case '00101101'
 sboxes= '11011000';
	case '00101110'
 sboxes= '00110001';
	case '00101111'
 sboxes= '00010101';
	 %forth row
	case '00110000'
 sboxes= '00000100';
	case '00110001'
 sboxes= '11000111';
	case '00110010'
 sboxes= '00100011';
	case '00110011'
 sboxes= '11000011';
	case '00110100'
 sboxes= '00011000';
	case '00110101'
 sboxes= '10010110';
	case '00110110'
 sboxes= '00000101';
	case '00110111'
 sboxes= '10011010';
	case '00111000'
 sboxes= '00000111';
	case '00111001'
 sboxes= '00010010';
	case '00111010'
 sboxes= '10000000';
	case '00111011'
 sboxes= '11100010';
	case '00111100'
 sboxes= '11101011';
	case '00111101'
 sboxes= '00100111';
	case '00111110'
 sboxes= '10110010';
	case '00111111'
 sboxes= '01110101';
	%fifth row
	case '01000000'
 sboxes= '00001001';
	case '01000001'
 sboxes= '10000011';
	case '01000010'
 sboxes= '00101100';
	case '01000011'
 sboxes= '00011010';
	case '01000100'
 sboxes= '00011011';
	case '01000101'
 sboxes= '01101110';
	case '01000110'
 sboxes= '01011010';
	case '01000111'
 sboxes= '10100000';
	case '01001000'
 sboxes= '01010010';
	case '01001001'
 sboxes= '00111011';
	case '01001010'
 sboxes= '11010110';
	case '01001011'
 sboxes= '10110011';
	case '01001100'
 sboxes= '00101001';
	case '01001101'
 sboxes= '11100011';
	case '01001110'
 sboxes= '00101111';
	case '01001111'
 sboxes= '10000100';
	%sixth row
	case '01010000'
 sboxes= '01010011';
	case '01010001'
 sboxes= '11010001';
	case '01010010'
 sboxes= '00000000';
	case '01010011'
 sboxes= '11101101';
	case '01010100'
 sboxes= '00100000';
	case '01010101'
 sboxes= '11111100';
	case '01010110'
 sboxes= '10110001';
	case '01010111'
 sboxes= '01011011';
	case '01011000'
 sboxes= '01101010';
	case '01011001'
 sboxes= '11001011';
	case '01011010'
 sboxes= '10111110';
	case '01011011'
 sboxes= '00111001';
	case '01011100'
 sboxes= '01001010';
	case '01011101'
 sboxes= '01001100';
	case '01011110'
 sboxes= '01011000';
	case '01011111'
 sboxes= '11001111';
	%seventh row
	case '01100000'
 sboxes= '11010000';
	case '01100001'
 sboxes= '11101111';
	case '01100010'
 sboxes= '10101010';
	case '01100011'
 sboxes= '11111011';
	case '01100100'
 sboxes= '01000011';
	case '01100101'
 sboxes= '01001101';
	case '01100110'
 sboxes= '00110011';
	case '01100111'
 sboxes= '10000101';
	case '01101000'
 sboxes= '01000101';
	case '01101001'
 sboxes= '11111001';
	case '01101010'
 sboxes= '00000010';
	case '01101011'
 sboxes= '01111111';
	case '01101100'
 sboxes= '01010000';
	case '01101101'
 sboxes= '00111100';
	case '01101110'
 sboxes= '10011111';
	case '01101111'
 sboxes= '10101000';
	%eighth row
	case '01110000'
 sboxes= '01010001';
	case '01110001'
 sboxes= '10100011';
	case '01110010'
 sboxes= '01000000';
	case '01110011'
 sboxes= '10001111';
	case '01110100'
 sboxes= '10010010';
	case '01110101'
 sboxes= '10011101';
	case '01110110'
 sboxes= '00111000';
	case '01110111'
 sboxes= '11110101';
	case '01111000'
 sboxes= '10111100';
	case '01111001'
 sboxes= '10110110';
	case '01111010'
 sboxes= '11011010';
	case '01111011'
 sboxes= '00100001';
	case '01111100'
 sboxes= '00010000';
	case '01111101'
 sboxes= '11111111';
	case '01111110'
 sboxes= '11110011';
	case '01111111'
 sboxes= '11010010';
	%ninth row
	case '10000000'
 sboxes= '11001101';
	case '10000001'
 sboxes= '00001100';
	case '10000010'
 sboxes= '00010011';
	case '10000011'
 sboxes= '11101100';
	case '10000100'
 sboxes= '01011111';
	case '10000101'
 sboxes= '10010111';
	case '10000110'
 sboxes= '01000100';
	case '10000111'
 sboxes= '00010111';
	case '10001000'
 sboxes= '11000100';
	case '10001001'
 sboxes= '10100111';
	case '10001010'
 sboxes= '01111110';
	case '10001011'
 sboxes= '00111101';
	case '10001100'
 sboxes= '01100100';
	case '10001101'
 sboxes= '01011101';
	case '10001110'
 sboxes= '00011001';
	case '10001111'
 sboxes= '01110011';
	%tenth row
	case '10010000'
 sboxes= '01100000';
	case '10010001'
 sboxes= '10000001';
	case '10010010'
 sboxes= '01001111';
	case '10010011'
 sboxes= '11011100';
	case '10010100'
 sboxes= '00100010';
	case '10010101'
 sboxes= '00101010';
	case '10010110'
 sboxes= '10010000';
	case '10010111'
 sboxes= '10001000';
	case '10011000'
 sboxes= '01000110';
	case '10011001'
 sboxes= '11101110';
	case '10011010'
 sboxes= '10111000';
	case '10011011'
 sboxes= '00010100';
	case '10011100'
 sboxes= '11011110';
	case '10011101'
 sboxes= '01011110';
	case '10011110'
 sboxes= '00001011';
	case '10011111'
 sboxes= '11011011';
	%eleventh row
	case '10100000'
 sboxes= '11100000';
	case '10100001'
 sboxes= '00110010';
	case '10100010'
 sboxes= '00111010';
	case '10100011'
 sboxes= '00001010';
	case '10100100'
 sboxes= '01001001';
	case '10100101'
 sboxes= '00000110';
	case '10100110'
 sboxes= '00100100';
	case '10100111'
 sboxes= '01011100';
	case '10101000'
 sboxes= '11000010';
	case '10101001'
 sboxes= '11010011';
	case '10101010'
 sboxes= '10101100';
	case '10101011'
 sboxes= '01100010';
	case '10101100'
 sboxes= '10010001';
	case '10101101'
 sboxes= '10010101';
	case '10101110'
 sboxes= '11100100';
	case '10101111'
 sboxes= '01111001';
	%twelveth row
	case '10110000'
 sboxes= '11100111';
	case '10110001'
 sboxes= '11001000';
	case '10110010'
 sboxes= '00110111';
	case '10110011'
 sboxes= '01101101';
	case '10110100'
 sboxes= '10001101';
	case '10110101'
 sboxes= '11010101';
	case '10110110'
 sboxes= '01001110';
	case '10110111'
 sboxes= '10101001';
	case '10111000'
 sboxes= '01101100';
	case '10111001'
 sboxes= '01010110';
	case '10111010'
 sboxes= '11110100';
	case '10111011'
 sboxes= '11101010';
	case '10111100'
 sboxes= '01100101';
	case '10111101'
 sboxes= '01111010';
	case '10111110'
 sboxes= '10101110';
	case '10111111'
 sboxes= '00001000';
	%thirteenth row
	case '11000000'
 sboxes= '10111010';
	case '11000001'
 sboxes= '01111000';
	case '11000010'
 sboxes= '00100101';
	case '11000011'
 sboxes= '00101110';
	case '11000100'
 sboxes= '00011100';
	case '11000101'
 sboxes= '10100110';
	case '11000110'
 sboxes= '10110100';
	case '11000111'
 sboxes= '11000110';
	case '11001000'
 sboxes= '11101000';
	case '11001001'
 sboxes= '11011101';
	case '11001010'
 sboxes= '01110100';
	case '11001011'
 sboxes= '00011111';
	case '11001100'
 sboxes= '01001011';
	case '11001101'
 sboxes= '10111101';
	case '11001110'
 sboxes= '10001011';
	case '11001111'
 sboxes= '10001010';
	%forteenth row
	case '11010000'
 sboxes= '01110000';
	case '11010001'
 sboxes= '00111110';
	case '11010010'
 sboxes= '10110101';
	case '11010011'
 sboxes= '01100110';
	case '11010100'
 sboxes= '01001000';
	case '11010101'
 sboxes= '00000011';
	case '11010110'
 sboxes= '11110110';
	case '11010111'
 sboxes= '00001110';
	case '11011000'
 sboxes= '01100001';
	case '11011001'
 sboxes= '00110101';
	case '11011010'
 sboxes= '01010111';
	case '11011011'
 sboxes= '10111001';
	case '11011100'
 sboxes= '10000110';
	case '11011101'
 sboxes= '11000001';
	case '11011110'
 sboxes= '00011101';
	case '11011111'
 sboxes= '10011110';
	%fifteenth row
	case '11100000'
 sboxes= '11100001';
	case '11100001'
 sboxes= '11111000';
	case '11100010'
 sboxes= '10011000';
	case '11100011'
 sboxes= '00010001';
	case '11100100'
 sboxes= '01101001';
	case '11100101'
 sboxes= '11011001';
	case '11100110'
 sboxes= '10001110';
	case '11100111'
 sboxes= '10010100';
	case '11101000'
 sboxes= '10011011';
	case '11101001'
 sboxes= '00011110';
	case '11101010'
 sboxes= '10000111';
	case '11101011'
 sboxes= '11101001';
	case '11101100'
 sboxes= '11001110';
	case '11101101'
 sboxes= '01010101';
	case '11101110'
 sboxes= '00101000';
	case '11101111'
 sboxes= '11011111';
	%sixteenth row
	case '11110000'
 sboxes= '10001100';
	case '11110001'
 sboxes= '10100001';
	case '11110010'
 sboxes= '10001001';
	case '11110011'
 sboxes= '00001101';
	case '11110100'
 sboxes= '10111111';
	case '11110101'
 sboxes= '11100110';
	case '11110110'
 sboxes= '01000010';
	case '11110111'
 sboxes= '01101000';
	case '11111000'
 sboxes= '01000001';
	case '11111001'
 sboxes= '10011001';
	case '11111010'
 sboxes= '00101101';
	case '11111011'
 sboxes= '00001111';
	case '11111100'
 sboxes= '10110000';
	case '11111101'
 sboxes= '01010100';
	case '11111110'
 sboxes= '10111011';
	case '11111111'
 sboxes= '00010110';
end

sboxees=bin2dec(sboxes);
end


In [48]:
%%file extract_last_round.m
%%%%%%%
% extract traces corresponding to the last round
%%%%%%%
function [traces_last] = extract_last_round(traces)
    % extract traces corresponding to the last round
    traces_last = zeros(20000, 3200 - 2700 + 1);
    for i = 1:20000
        traces_last(i,:) = traces(i, 2700:3200);
    end
end

In [49]:
%%file key_schu.m
function new_key = key_schu(key,round)
    Rcon=[1,2,4,8,16,32,64,128,27,54;0,0,0,0,0,0,0,0,0,0;0,0,0,0,0,0,0,0,0,0;0,0,0,0,0,0,0,0,0,0];
    M=key;
    new_key=key;
    save=Subbytes(key(1,4));
    for i= 2:4
        M(i-1,4)=Subbytes(M(i,4));
    end
    M(4,4)=save;

    for j=1:4
        if j==1
            for k=1:4
            new_key(k,j)= bitxor(bitxor(key(k,j),M(k,4)),Rcon(k,round));
            end
        else
            for l=1:4
            new_key(l,j)=bitxor(key(l,j),new_key(l,j-1));
            end
        end
    end
end